In [ ]:
! pip install datasets transformers[sentencepiece]
import importlib

In [ ]:
import                          torch, os

import numpy                    as np
import pandas                   as pd
import matplotlib.pyplot        as plt

from google.colab               import drive
from datasets                   import DatasetDict
from datasets                   import load_from_disk
from sklearn.preprocessing      import MultiLabelBinarizer

In [ ]:
print("Working Directory:", os.getcwd())

# Mount Google Drive
drive.mount('/content/drive')

# Define the folder path
folder_path = "/content/drive/MyDrive/NLP_class"

# Change the working directory to a specific path
os.chdir(folder_path)
# Print the updated working directory
print("Updated Working Directory:", os.getcwd())

In [ ]:
import utils
importlib.reload(utils)
from utils import df_to_DatasetDict, MyDataset, Trainer, Evaluator

# Dataset

In [ ]:
# Create folder to save dataset
dataset_dir = os.path.join(folder_path,'dataset')
os.makedirs(dataset_dir, exist_ok=True)

# output dir
out_dir = os.path.join(folder_path,'results')
os.makedirs(os.path.join(folder_path,'results'), exist_ok=True)

In [ ]:
# data_path = "/content/drive/MyDrive/NLP_class/train.csv"
# df = pd.read_csv(data_path)
# # combining 'title' and 'abstract' column to| get more context
# df['text'] = df['TITLE'] + "."+df['ABSTRACT']
# # dropping useless features/columns
# df.drop(labels=['TITLE', 'ABSTRACT', 'ID'], axis=1, inplace=True)
# # rearranging columns
# df = df[['text', 'Computer Science', 'Physics', 'Mathematics', 'Statistics',
#                      'Quantitative Biology', 'Quantitative Finance',]]

# # Convert the label columns to a list of labels
# label_columns = ["Computer Science", "Physics", "Mathematics", "Statistics", "Quantitative Biology", "Quantitative Finance"]
# labels = df[label_columns].values.tolist()

# # Combine the text column with the list of labels
# df["labels"] = labels

# # Drop the original label columns
# df.drop(columns=label_columns, inplace=True)

# # Now df contains the "text" column and the "labels" column with lists of 1s and 0s

# df.head()

In [ ]:
# train_size = 0.8
# val_size   = 0.2
# test_size = None
# dataset_dir = dataset_dir
# df_to_DatasetDict(df,
#                   train_size,
#                   val_size,
#                   dataset_dir = dataset_dir,
#                   frac=1,
#                   random_state=200)

# dataset = load_from_disk(os.path.join(dataset_dir))
# dataset

In [ ]:
data_path = "https://raw.githubusercontent.com/laxmimerit/All-CSV-ML-Data-Files-Download/master/IMDB-Movie-Data.csv"
usecols=["Description", "Genre"]
df = pd.read_csv(data_path, usecols=usecols)
# df.head()

multilabel = MultiLabelBinarizer()

df['Genre'] = df['Genre'].str.split(',')
labels = multilabel.fit_transform(df['Genre']).astype('float32')
df['labels'] = list(labels)

# Drop 'Genre' column
df.drop(columns=['Genre'], inplace=True)

# Change the name of 'Description' column
df.rename(columns={'Description': 'text'}, inplace=True)  # Change column 'A' to 'New_A'

df.head()

In [ ]:
# df.columns

In [ ]:
train_size = 0.7
val_size   = 0.2
test_size  = 0.1
dataset_dir = dataset_dir
df_to_DatasetDict(df,
                  train_size,
                  val_size,
                  test_size,
                  dataset_dir = dataset_dir)

dataset = load_from_disk(os.path.join(dataset_dir))
dataset

In [ ]:
num_classes = len(dataset['train']['labels'][0])
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
trainer = Trainer(
                  dataset_dir,
                  out_dir,
                  num_classes,
                  patience = 10,
                  model_ckpt = "distilbert-base-uncased",
                  problem_type = "multi_label_classification",
                  max_len = 256,
                  optimizer = 'Adam',
                  init_lr = 1e-5,
                  weight_decay = 0,
                  scheduler_type = "linear",
                  num_epochs = 1,
                  train_bs = 32,
                  val_bs = 32,
                  device = device,
                  clf_thrshold = 0.3)

In [ ]:
trainer.train()

In [ ]:
# ckpt_name = 'dump/run_12/training_checkpoint.pth'
# ckpt = torch.load(os.path.join(out_dir, ckpt_name))
# ckpt.keys()

# Evaluation

In [ ]:
# dataset_dir = os.path.join(folder_path,'dataset')
# out_dir = os.path.join(folder_path,'results')
# model_dir = os.path.join(out_dir, 'dump/run_7')

# num_classes = len(dataset['train']['labels'][0])
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# evaluator = Evaluator(dataset_dir,
#                       model_dir = model_dir,
#                       num_classes = num_classes,
#                       ckpt_name    = "training_checkpoint.pth",
#                       model_ckpt   = "distilbert-base-uncased",
#                       problem_type = "multi_label_classification",
#                       device       = device,
#                       clf_thrshold = 0.3)

In [ ]:
# evaluator.evaluate()

# Prediction